In [ ]:
from phi import prepare_phi, inference

model, tokenizer = prepare_phi(trainable=False)

In [ ]:
from askllm import parse_result, makeprompt
import json

def call(model, tokenizer, sysprompt, prompt):
    chat = (f"<|im_start|>system<|im_sep|>{sysprompt}<|im_end|>"
            f"<|im_start|>user<|im_sep|>{prompt}<|im_end|>"
            f"<|im_start|>assistant<|im_sep|>")
    return inference(model, tokenizer, chat, 3000)[0]

problemfile = "test.txt"
problems=[]
rowcount=0  
with open(problemfile, "r") as f:
    next(f)
    for i, line in enumerate(f):
        problems.append(json.loads(line.strip()))

processed=0
correct = {
    "horn": {"sat": 0, "unsat": 0},
    "not_horn": {"sat": 0, "unsat": 0}
}
for problem in problems:
    sysprompt=None
    prompt=makeprompt(problem)
    result=call(model, tokenizer, sysprompt, prompt)
    processed+=1

    got = parse_result(result)
    sat = problem[4]
    horn = problem[3]
    print(f"got={got} sat={sat} horn={horn}")

    if got == sat:
        correct["horn" if horn else "not_horn"]["sat" if sat else "unsat"] += 1

print(f"\nSummary: processed={processed} correct={correct}")
